In [1]:
import torch
import torch.nn as nn
import numpy as np
from numpy.random import default_rng


def set_seed(seed=0):
    seed = 42069  # set a random seed for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [2]:
def init_weight(random=False):
    if random:
        rng = default_rng(0)
         # y0 dim: (1, 2)
        w1 = rng.standard_normal((3, 2), dtype='f')
        # y1 dim: (1, 3)
        w2 = rng.standard_normal((3, 3), dtype='f')
        # y2 dim: (1, 3)
        w3 = rng.standard_normal((2, 3), dtype='f')
    else:
        # y0 dim: (1, 2)
        w1 = np.array([[0.2, 0.3],
                       [0.4, 0.2],
                       [0.3, 0.4]], dtype='f')
        # y1 dim: (1, 3)
        w2 = np.array([[0.2, 0.3, 0.4],
                       [0.4, 0.2, 0.3],
                       [0.3, 0.4, 0.2]], dtype='f')
        # y2 dim: (1, 3)
        w3 = np.array([[0.2, 0.3, 0.4],
                       [0.4, 0.2, 0.3]], dtype='f')
        # y3 dim: (1, 2)
    return w1, w2, w3


class CMlp(nn.Module):

    def __init__(self, encrypt=False, weight_random=False):
        super(CMlp, self).__init__()
        w1, w2, w3 = init_weight(random=weight_random)
        self.fc1 = nn.Linear(2, 3, False)
        self.fc2 = nn.Linear(3, 3, False)
        self.fc3 = nn.Linear(3, 2, False)

        if encrypt:
            noise_rng = default_rng(1)
            self.r1 = noise_rng.standard_normal((3, 1), dtype='f')
            self.r2 = noise_rng.standard_normal((3, 1), dtype='f')
            # self.r3 = noise_rng.standard_normal((2, 1), dtype='f')
            self.r3 = noise_rng.standard_normal((1, 1), dtype='f')

            self.fc1.weight.data = torch.from_numpy(w1 * self.r1)
            self.fc2.weight.data = torch.from_numpy(w2 * self.r2 / self.r1.transpose())
            self.fc3.weight.data = torch.from_numpy(w3 * self.r3 / self.r2.transpose())
        else:
            self.fc1.weight.data = torch.from_numpy(w1)
            self.fc2.weight.data = torch.from_numpy(w2)
            self.fc3.weight.data = torch.from_numpy(w3)

        self.y2 = None
        self.y3 = None
        self.alpha = None

    def forward(self, x):
        y1 = self.fc1(x)
        self.y2 = self.fc2(y1)
        self.alpha = self.y2.sum()
        self.y3 = self.fc3(self.y2)
        self.y3.retain_grad()
        return self.y3

In [3]:
# setup gpu or cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'

x = torch.tensor([[0.2, 0.3]], device=device)
y_hat = torch.tensor([[0.5, 0.5]], device=device)

net = CMlp(weight_random=True).to(device)
print('----------- plaintext weight ---------------')
for p in net.parameters():
    print(p.data)
y = net(x)
print('y: ', y)
criterion = nn.MSELoss()
loss = criterion(y, y_hat)
loss.backward(retain_graph=True)
print('----------- plaintext grad -----------------')
for p in net.parameters():
    print(p.grad)
w_gradlist = [p.grad for p in net.parameters()]
print('----------- ciphertext weight ---------------')
net_c = CMlp(encrypt=True, weight_random=True).to(device)
for p in net_c.parameters():
    print(p.data)
y_c = net_c(x)
c_loss = criterion(y_c, y_hat)
c_loss.backward(retain_graph=True)
print('----------- ciphertext grad ---------------')
for p in net_c.parameters():
    print(p.grad)
c_w_gradlist = [p.grad.detach().clone() for p in net_c.parameters()]

print('Get yc: ', y_c)
r3 = torch.from_numpy(net_c.r3.transpose()).to(device)
print('Get yc from y: ', y  * r3)
print('Ly derivative')
print(y - y_hat)
print(y.grad)
y_grad = y.grad.clone()
print('Lhaty derivative')
print(y_c - y_hat)
print(y_c.grad)
y_c_grad = y_c.grad

----------- plaintext weight ---------------
tensor([[ 1.1176, -1.3871],
        [-0.4266, -0.8036],
        [ 0.6014, -0.0750]], device='cuda:0')
tensor([[ 0.0597, -0.0320, -0.1855],
        [ 1.2048,  0.7775, -1.3583],
        [ 0.7698, -0.8702,  1.0998]], device='cuda:0')
tensor([[-0.9585, -1.2749, -1.3653],
        [-1.4743,  0.4335, -0.3281]], device='cuda:0')
y:  tensor([[ 0.4749, -0.3197]], device='cuda:0', grad_fn=<MmBackward>)
----------- plaintext grad -----------------
tensor([[-0.0165, -0.0248],
        [-0.1109, -0.1664],
        [ 0.1088,  0.1632]], device='cuda:0')
tensor([[-0.2374, -0.4023,  0.1205],
        [ 0.0623,  0.1056, -0.0316],
        [-0.0584, -0.0990,  0.0297]], device='cuda:0')
tensor([[ 4.8124e-04,  1.5515e-02, -6.1016e-03],
        [ 1.5730e-02,  5.0711e-01, -1.9944e-01]], device='cuda:0')
----------- ciphertext weight ---------------
tensor([[ 1.9325, -2.3985],
        [ 0.6093,  1.1479],
        [ 0.6181, -0.0771]], device='cuda:0')
tensor([[ 0.0650,  0

$ \frac{\partial \widehat{L}}{\partial \widehat{W}^{(l)}} = ( \frac{\partial L}{\partial y^{(L)}} + (r^L - 1) \cdot y^{L}) \frac{\partial \widehat{y}^{(L)}}{\partial \widehat{W}^{(l)}} $

In [4]:
optim = torch.optim.Optimizer(net.parameters(), {})
optim_c = torch.optim.Optimizer(net_c.parameters(), {})
optim_c.zero_grad()

In [5]:
n3 = r3.clone()
r_L = (n3 - 1) * y
print(r_L)
t = y_grad + r_L
y_c.backward(t, retain_graph=True)

tensor([[-1.3393,  0.9017]], device='cuda:0', grad_fn=<MulBackward0>)


### get grad $\frac{\partial \widehat{L}}{\partial \widehat{W}^{(l)}}$

In [6]:
for p in net_c.parameters():
    print(p.grad)

tensor([[-0.7627, -1.1441],
        [-0.0652, -0.0977],
        [ 0.2168,  0.3252]], device='cuda:0')
tensor([[  0.3823,  -0.5352,  -0.1154],
        [ 14.3215, -20.0486,  -4.3217],
        [  2.6677,  -3.7344,  -0.8050]], device='cuda:0')
tensor([[ 0.0493,  0.0634, -0.1385],
        [-0.0030, -0.0038,  0.0083]], device='cuda:0')


In [7]:
print(*c_w_gradlist, sep='\n')

tensor([[-0.7627, -1.1441],
        [-0.0652, -0.0977],
        [ 0.2168,  0.3252]], device='cuda:0')
tensor([[  0.3823,  -0.5352,  -0.1154],
        [ 14.3215, -20.0486,  -4.3217],
        [  2.6677,  -3.7344,  -0.8050]], device='cuda:0')
tensor([[ 0.0493,  0.0634, -0.1385],
        [-0.0030, -0.0038,  0.0083]], device='cuda:0')


$ \frac{\partial \widehat{L}}{\partial \widehat{W}^{(l)}} = \frac{r^L}{R_l} \frac{\partial L}{\partial {W}^{(l)}}+ (r^L - 1) \cdot y^{L} \frac{\partial \widehat{y}^{(L)}}{\partial \widehat{W}^{(l)}} $

In [8]:
denoise = [1 / net_c.r1, net_c.r1.transpose() / net_c.r2, net_c.r2.transpose() / net_c.r3]
print(*denoise, sep='\n') # noise for w_ij in each layer

# left of addition
l_tensor_list =[]
for x, n in zip(w_gradlist, denoise):
    print(x.mul(torch.from_numpy(n).to(device)).mul(torch.from_numpy(net_c.r3).to(device)))
    l_tensor_list.append(x.mul(torch.from_numpy(n).to(device)).mul(torch.from_numpy(net_c.r3).to(device)))

optim_c.zero_grad()
y_c.backward(r_L, retain_graph=True)

# right of addition
r_tensor_list = [p.grad.detach().clone() for p in net_c.parameters()]

[[ 0.57833433]
 [-0.7000578 ]
 [ 0.9730042 ]]
[[  0.9187248   -0.75898033   0.54607177]
 [ 23.013092   -19.011662    13.678524  ]
 [  4.1443925   -3.4237807    2.4633443 ]]
[[-1.0340182  -0.0412799  -0.22922012]]
tensor([[ 0.0174,  0.0261],
        [-0.1414, -0.2121],
        [-0.1927, -0.2891]], device='cuda:0')
tensor([[ 0.3970, -0.5558, -0.1198],
        [-2.6093,  3.6527,  0.7874],
        [ 0.4406, -0.6167, -0.1329]], device='cuda:0')
tensor([[ 0.0009,  0.0012, -0.0025],
        [ 0.0296,  0.0381, -0.0832]], device='cuda:0')


### restore ciphertext grad in left of the equation

In [9]:
for l, r in zip(l_tensor_list, r_tensor_list):
    print(l + r)

tensor([[-0.7627, -1.1441],
        [-0.0652, -0.0977],
        [ 0.2168,  0.3252]], device='cuda:0')
tensor([[  0.3823,  -0.5352,  -0.1154],
        [ 14.3215, -20.0487,  -4.3217],
        [  2.6677,  -3.7344,  -0.8050]], device='cuda:0')
tensor([[ 0.0493,  0.0634, -0.1385],
        [-0.0030, -0.0038,  0.0083]], device='cuda:0')


$ \frac{R_l}{r^L} (\frac{\partial \widehat{L}}{\partial \widehat{W}^{(l)}} - (r^L - 1) \cdot y^{L} \frac{\partial \widehat{y}^{(L)}}{\partial \widehat{W}^{(l)}})  =  \frac{\partial L}{\partial {W}^{(l)}} $

In [10]:
denoise_cipher = [net_c.r1, net_c.r2 / net_c.r1.transpose(), net_c.r3 / net_c.r2.transpose()]

for x, y, n in zip(c_w_gradlist, r_tensor_list, denoise_cipher):
    z = x - y
    print(z.mul(torch.from_numpy(n).to(device)).mul(torch.from_numpy(1 / net_c.r3).to(device)))



tensor([[-0.0165, -0.0248],
        [-0.1109, -0.1664],
        [ 0.1088,  0.1632]], device='cuda:0')
tensor([[-0.2374, -0.4023,  0.1205],
        [ 0.0623,  0.1056, -0.0316],
        [-0.0584, -0.0990,  0.0297]], device='cuda:0')
tensor([[ 4.8124e-04,  1.5514e-02, -6.1016e-03],
        [ 1.5730e-02,  5.0711e-01, -1.9944e-01]], device='cuda:0')
